In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm


In [2]:
bag_of_words_top_1000_topics = pd.read_feather('../../data/transform/bag_of_words_top_1000_topics.feather')

In [4]:
for i in bag_of_words_top_1000_topics.columns:
    print(i)

track_id_
title_
song_id_
release_
artist_id_
artist_mbid_
artist_name_
duration_
artist_familiarity_
artist_hotttnesss_
release_year_
genre_
love
know
like
que
time
come
de
go
oh
la
one
get
see
got
never
feel
want
let
make
way
say
take
would
ca
day
baby
away
life
yeah
back
heart
could
night
need
look
en
eyes
things
world
'cause
gonna
die
right
tell
think
un
el
man
live
give
te
try
tu
se
keep
dream
well
mind
still
us
long
mi
girl
wanna
find
fall
around
good
little
turn
hand
nothing
light
said
call
every
always
e
waiting
leave
home
hold
run
es
walk
ever
lies
face
end
hear
gone
head
believe
everything
ya
cry
alone
yo
hey
le
something
si
inside
du
words
stay
place
much
change
ich
god
stand
better
last
friend
soul
another
je
left
new
stop
tonight
burn
sun
por
lo
play
thought
sing
show
lost
people
made
break
und
start
really
chorus
hard
sky
old
da
even
boy
please
song
care
without
hope
watch
pain
amor
move
wrong
put
free
gotta
name
et
blue
con
dance
talk
seems
mine
dark
na
far
tears
true
fi

In [4]:
#scale by word count
bag_of_words_top_1000_topics['sum'] = bag_of_words_top_1000_topics.iloc[:,12:900].sum(axis=1)

bag_of_words_top_1000_topics['love'] = bag_of_words_top_1000_topics['love']/bag_of_words_top_1000_topics['sum']
bag_of_words_top_1000_topics['religion'] = bag_of_words_top_1000_topics['religion']/bag_of_words_top_1000_topics['sum']
bag_of_words_top_1000_topics['death'] = bag_of_words_top_1000_topics['death']/bag_of_words_top_1000_topics['sum']
bag_of_words_top_1000_topics.drop('sum', axis=1, inplace=True)

In [5]:
rank_order = np.array(range(len(bag_of_words_top_1000_topics)))/(len(bag_of_words_top_1000_topics)-1)

bag_of_words_top_1000_topics = bag_of_words_top_1000_topics.sort_values(by='religion')
bag_of_words_top_1000_topics = bag_of_words_top_1000_topics.reset_index(drop=True)
bag_of_words_top_1000_topics['religion'] = rank_order

bag_of_words_top_1000_topics = bag_of_words_top_1000_topics.sort_values(by='death')
bag_of_words_top_1000_topics = bag_of_words_top_1000_topics.reset_index(drop=True)
bag_of_words_top_1000_topics['death'] = rank_order

bag_of_words_top_1000_topics = bag_of_words_top_1000_topics.sort_values(by='love')
bag_of_words_top_1000_topics = bag_of_words_top_1000_topics.reset_index(drop=True)
bag_of_words_top_1000_topics['love'] = rank_order

In [8]:
bag_of_words_top_1000_topics.to_feather('../../data/clean/bag_of_words_top_1000_weightedtopics.feather')

In [11]:
bag_with_year = bag_of_words_top_1000_topics[bag_of_words_top_1000_topics['release_year_'].notnull()].reset_index()

weights_yearly_clean = {}
weights_yearly_clean['love'] = [0] * len(bag_with_year['release_year_'].unique())
weights_yearly_clean['death'] = [0] * len(bag_with_year['release_year_'].unique())
weights_yearly_clean['religion'] = [0] * len(bag_with_year['release_year_'].unique())
weights_yearly_clean['year'] = [0] * len(bag_with_year['release_year_'].unique())

for i, val in enumerate(bag_with_year['release_year_'].unique()):
    temp = bag_with_year[bag_with_year['release_year_']==val]
    weights_yearly_clean['year'][i] = val
    weights_yearly_clean['love'][i] = temp.love.mean()
    weights_yearly_clean['death'][i] = temp.death.mean()
    weights_yearly_clean['religion'][i] = temp.religion.mean()

topics_per_year = pd.DataFrame(weights_yearly_clean).sort_values(by='year').reset_index(drop=True)

In [24]:
artist_loc = pd.read_feather('../../data/transform/artist_location.feather')
bag_with_artistloc = bag_of_words_top_1000_topics.set_index('artist_id_').join(artist_loc.set_index('artist_id'), how='inner').reset_index()
c = bag_with_artistloc.columns.values
c[0] = 'artist_id'
bag_with_artistloc.columns = c

weights_artist_clean = {}
weights_artist_clean['love'] = [0] * len(bag_with_artistloc.artist_id.unique())
weights_artist_clean['death'] = [0] * len(bag_with_artistloc.artist_id.unique())
weights_artist_clean['religion'] = [0] * len(bag_with_artistloc.artist_id.unique())
weights_artist_clean['artist_id'] = [0] * len(bag_with_artistloc.artist_id.unique())
weights_artist_clean['artist_name'] = [0] * len(bag_with_artistloc.artist_id.unique())
weights_artist_clean['longitude'] = [0] * len(bag_with_artistloc.artist_id.unique())
weights_artist_clean['latitude'] = [0] * len(bag_with_artistloc.artist_id.unique())

for i, val in enumerate(bag_with_artistloc.artist_id.unique()):
    temp = bag_with_artistloc[bag_with_artistloc.artist_id==val]
    weights_artist_clean['artist_id'][i] = val
    weights_artist_clean['artist_name'][i] = temp.artist_name.iloc[0]
    weights_artist_clean['longitude'][i] = temp.longitude.iloc[0]
    weights_artist_clean['latitude'][i] = temp.latitude.iloc[0]
    weights_artist_clean['love'][i] = temp.love.mean()
    weights_artist_clean['death'][i] = temp.death.mean()
    weights_artist_clean['religion'][i] = temp.religion.mean()

topics_per_artist = pd.DataFrame(weights_artist_clean).sort_values(by='artist_name').reset_index(drop=True)

In [25]:
topics_per_artist.head()

,artist_id,artist_name,death,latitude,longitude,love,religion
0,ARBRIE41187FB4D1BE,10 Years,0.838404,30.97420,-91.52382,0.236303,0.356802
1,ARY8OMX1187FB58D9B,101,0.135087,37.27188,-119.27023,0.882896,0.199418
2,ARUBDBK1187FB4D1B3,10cc,0.479725,53.47960,-2.24881,0.570607,0.532486
3,AR62RK81187B994208,11/5_ Billy,0.770147,38.89910,-77.02900,0.303924,0.728144
4,AR5VDH81187B98FB66,113 feat. Le Rat Lucciano & Booba,0.792706,48.83515,2.24102,0.336460,0.741132


In [27]:
topics_per_artist.to_feather('../../data/transform/artist_topic_weights.feather')
topics_per_year.to_feather('../../data/transform//year_topic_weights.feather')